# EPiC Pandas: Excel Pivot Calculations in Pandas

In [1]:
## Library import
import pandas as pd
import numpy as np

## General thoughts on data

- Logic and data: should, in principle, be seperated
    - Excel's biggest single problem - but that is a different question !
- __TLF-format__: "Tidy Long Format" - data should be available, on the most granular level, in a "tidy long format". That is to say that all the __features__ (also called __dimensions__ in some contexts) describing a data point (aka __metric__) should be available as distinct columns in the table or data frame
    - Features are categorical variables
        - somteimes "binning" is needed, e.g. creating age-groups based on value metric "age"
    - Number of features determines the __dimensionality__ of the data
    - The combination of categorical values describes a data point
        - Important question: only UNIQUE data point descriptions in data ?
            - multi-indexing in Pandas runs into problems if not; but the problem can be solved by a "cautionary" GroupBy on all features to make sure that eventuel rows with the same feature combination are aggregated

In [2]:
#1: This looks like "typical data" - but it is NOT in tidy long format (non-TDF)
df_imp_simp_not_tdf = pd.read_csv('simple_data_NOT_TLF.csv', encoding = 'latin1', sep=';', decimal =',')
df_imp_simp_not_tdf.head(5)

,STORE,PRODUCT_GROUP,01_JAN,02_FEB,03_MAR,04_APR,05_MAY,06_JUN
0,hyper,appliance,259,783,386,325,914,444
1,hyper,electronics,519,516,599,324,892,366
2,hyper,hygiene,279,256,258,621,452,754
3,mega,appliance,735,204,923,717,978,933
4,mega,electronics,907,976,204,563,651,501


The "correct" TDF-format of this data would rather need to look like this:

In [3]:
df_imp_simp_tdf = pd.read_csv('simple_data_TLF.csv', encoding = 'latin1', sep=';', decimal =',')
df_imp_simp_tdf.head(5)

,MONTH,STORE,PRODUCT_GROUP,REVENUE
0,01_JAN,mega,appliance,735
1,01_JAN,hyper,appliance,259
2,01_JAN,super,appliance,659
3,01_JAN,mega,electronics,907
4,01_JAN,hyper,electronics,519


The example should make clear where the problem lies: in the NON-TDF-format, the MONTH-__feature__ is organized along columns  and the nature of the __metric__ being REVENUE is only implicitly stated.

## Why Pivot Tables ?

Pivot tables are a widely used tool to establish an overview over large and multi-dimensional data sets. They are very useful because:
- Pivot tables allow a quick overview over __large and multi-dimensional datasets__ by aggregating information along the desired axis of analysis (aka: the features that represent the dimensions of the data we want to look at)
    - The data can easily be hierarchisized  both along rows and columns
- Pivot tables provide great input for visualizations
- Pivot tables are often the first step in the data processing pipeline from very granular data to further processing on the aggregated form of this data

## How does an Excel Pivot Table work ?

A typical Excel Pivot Table with all its constituant elements:

In [4]:
%%html
<img src="img/Excel_Pivot_Sample1.PNG">

### The defining elements of a pivot table

1. TLF-input
2. Filters set on the TLF dataset (optional)
3. Feature(s) from dataset to be aggregated along row
    * features can be stacked
    * Each combination of categorical values of selected row features creates 1 row
4. Features from dataset to be aggregated along columns
    * features can be stacked
    * each combination of categorical values of selected column feature creates 1 column
5. Values / metric to be aggregated
    * Aggegregation function used to aggegate the chosen metric
    * sub-aggregations eventually possible if features are stacked hierarchically in rows and/or columns

### Where Excel Pivot Assistant, well, excels:

- ad-hoch analysis on existing TLF-data
- automatic aggregation on chosen metric

### Shortcomings of Excel Pivot Assistant

- Data prep for TLF-format
- Excel Pivot table is __'irreversible'__
    - data modification or enhancement dos not work dynamically on pivot tables, e.g.
        - sum() on columns of pivot table
        - delta between columns of pivot table
    - modified or enhanced data does not 'reversed' to orginal TLF data
- Performance on large data sets
    - especially in the likely case that features in the TLF data set where enhanced based on a functional calculation (aka: feature value is result of Excel function)
        - VLOOKUP()-hell to perform 'binning'

# Python Pandas to the rescue....

How Pandas can address all limitations inherent to the Excel approach, espacially via the stack/unstack-function on multi-indexed data frames.

#### Structure of the sample data set we will work with:

In [5]:
%%html
<img src="img/sample_dataset_structure.PNG">

In [6]:
## Or the same in pandas: A neat TLF-format - a rare occurence in real life !
df_imp_struct = pd.read_csv('EPiC_dataset_structure.csv', encoding = 'latin1', sep=';', decimal =',')

In [7]:
df_imp_struct

,long_name,short_name,meaning,example_value
0,timestamp_1,ts1,date,15.12.2018
1,feature_0_lev1,f0_l1,year,2018
2,feature_0_lev2,f0_l2,month,12
3,feature_0_lev3,f0_l3,day,15
4,feature_1,f1,sales_rep,Wilbur
5,feature_2,f2,promo,yes
6,feature_3,f3,customer,metropolis
7,feature_4_hierarchy_lev1,f4_l1,ctry,Germany
8,feature_4_hierarchy_lev2,f4_l2,state,BadWurt
9,feature_4_hierarchy_lev3,f4_l3,city,Freiburg


## Multi-indexed data frame as basis for stacking/unstacking aka pivoting

#### Step1: import sampel data set in simple data frame format

In [8]:
df_imp = pd.read_csv('EPiC_dataset.csv', encoding = 'latin1', sep=';', decimal =',')
df_imp.head(3)

,ts1,f0_l1,f0_l2,f0_l3,f1,f2,f3,f4_l1,f4_l2,f4_l3,f5_l1,f5_l2,m1,m2,m3
0,15.12.2018,2018,12,15,Wilbur,yes,metropolis,Germany,BadWurt,Freiburg,stationary,pen,354,23.95,2.82
1,14.12.2018,2018,12,14,Wilbur,no,smartpoint,Germany,Berlin,Berlin,household,wipes,971,57.05,7.60
2,22.10.2018,2018,10,22,Greg,yes,thebarn,Germany,Bremen,Bremen,stationary,water colours,142,21.01,3.75


#### Step2: import sampel data set in multi-indexed data frame format

In [9]:
df_mi_imp = pd.read_csv('EPiC_dataset.csv',encoding = 'latin1', sep=';', 
                        decimal =',', index_col=[0,1,2,3,4,5,6,7,8,9,10,11])
df_mi_imp.head(3)

,,,,,,,,,,,,m1,m2,m3
ts1,f0_l1,f0_l2,f0_l3,f1,f2,f3,f4_l1,f4_l2,f4_l3,f5_l1,f5_l2,,,
15.12.2018,2018,12,15,Wilbur,yes,metropolis,Germany,BadWurt,Freiburg,stationary,pen,354,23.95,2.82
14.12.2018,2018,12,14,Wilbur,no,smartpoint,Germany,Berlin,Berlin,household,wipes,971,57.05,7.60
22.10.2018,2018,10,22,Greg,yes,thebarn,Germany,Bremen,Bremen,stationary,water colours,142,21.01,3.75


In [10]:
df_mi_imp.index.names

FrozenList(['ts1', 'f0_l1', 'f0_l2', 'f0_l3', 'f1', 'f2', 'f3', 'f4_l1', 'f4_l2', 'f4_l3', 'f5_l1', 'f5_l2'])

#### Step3: compare the difference

In [11]:
# width of the "simple" data frame
len(df_imp.columns)

15

In [12]:
# Fundamental difference:
# width of the multi-indexed data frame: only values are "real" data columns !
len(df_mi_imp.columns)

3

Learning: 'multi-indexing' means: the features are not seen as data themselves, but rather as meta-data organizing the core-data (aka metrics). Stack and unstack can be used to perform pivoting operations on this meta-data structure ! There is a .pivot() and a pivot_table()-method in pandas. Ironically enough, its scope is much more limited, as it performs on a simple data frame, not making full use of the strenght of multi-indexing. So we will solely concentrate on stack/unstack. If you want more information on the other methods:
- https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

### Finally: let's do some pivoting (stacking/unstacking)

Typical use case: __aggregate total units in data per state and product category__
- state = f4_l2 -> in rows
- category = f5_l1 -> in columns
- units = m1

In [13]:
df_w1 = df_mi_imp.unstack('f5_l1').fillna(0)
df_w1.head(5)

m1  \
f5_l1                                                                                       appliances   
ts1        f0_l1 f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2  f4_l3      f5_l2                       
01.10.2018 2018  10    1     Beth   yes metropolis Germany NRW    Cologne    vaccum cleaner      469.0   
                             Darma  yes care4you   Germany NRW    Düsseldorf mixer                98.0   
                             Harry  no  care4you   Germany Saxony Dresden    pen                   0.0   
                             Susan  yes dealhouse  Germany Berlin Berlin     ball-pen              0.0   
                             Wilbur no  care4you   Germany NRW    Essen      wipes                 0.0   

                                                                                                       \
f5_l1                                                                                       household   
ts1        f0_l1 f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2  f4_l3      f5_l2                      
01.10.2018 2018  10    1     Beth   yes metropolis Germany NRW    Cologne    vaccum cleaner       0.0   
                             Darma  yes care4you   Germany NRW    Düsseldorf mixer                0.0   
                             Harry  no  care4you   Germany Saxony Dresden    pen                  0.0   
                             Susan  yes dealhouse  Germany Berlin Berlin     ball-pen             0.0   
                             Wilbur no  care4you   Germany NRW    Essen      wipes              918.0   

                                                                                                        \
f5_l1                                                                                       stationary   
ts1        f0_l1 f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2  f4_l3      f5_l2                       
01.10.2018 2018  10    1     Beth   yes metropolis Germany NRW    Cologne    vaccum cleaner        0.0   
                             Darma  yes care4you   Germany NRW    Düsseldorf mixer                 0.0   
                             Harry  no  care4you   Germany Saxony Dresden    pen                 183.0   
                             Susan  yes dealhouse  Germany Berlin Berlin     ball-pen            523.0   
                             Wilbur no  care4you   Germany NRW    Essen      wipes                 0.0   

                                                                                                    m2  \
f5_l1                                                                                       appliances   
ts1        f0_l1 f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2  f4_l3      f5_l2                       
01.10.2018 2018  10    1     Beth   yes metropolis Germany NRW    Cologne    vaccum cleaner      97.67   
                             Darma  yes care4you   Germany NRW    Düsseldorf mixer               56.02   
                             Harry  no  care4you   Germany Saxony Dresden    pen                  0.00   
                             Susan  yes dealhouse  Germany Berlin Berlin     ball-pen             0.00   
                             Wilbur no  care4you   Germany NRW    Essen      wipes                0.00   

                                                                                                       \
f5_l1                                                                                       household   
ts1        f0_l1 f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2  f4_l3      f5_l2                      
01.10.2018 2018  10    1     Beth   yes metropolis Germany NRW    Cologne    vaccum cleaner      0.00   
                             Darma  yes care4you   Germany NRW    Düsseldorf mixer               0.00   
                             Harry  no  care4you   Germany Saxony Dresden    pen                 0.00   
                             Susan  yes dealhouse  Germany Berlin Berlin     ball-

Learnings on first use of .unstack():
- makes TLF-format __wider__ by moving selected feature to columns
- __.fillna()-parameter must not be forgotten__ to have defined entry for crossing with no data in the TLF
    - .fillna(0) mostly good idea due to the need for further calculations
- ....oups, there are all 3 metrics in the resulting data frame !
- ....oups, and what about the state-level aggregation ???
    - This is a shortcoming of Pandas vs. Excel: NO automatic aggregation is taking place when using .unstack(). Aggregation always needs to be done manually
    

### Stacking before unstacking: reduce data frame to series to turn the metrics info into just another categorical feature info


- maybe we should give the different metrics a meta-name

In [14]:
df_w2 = df_mi_imp
df_w2.columns.names = ['METRICS']
df_w2.head(3)
# This step can be skipped with a slight modification to the csv-import format.
# How to do this will be shown later in a more complex import example

,,,,,,,,,,,METRICS,m1,m2,m3
ts1,f0_l1,f0_l2,f0_l3,f1,f2,f3,f4_l1,f4_l2,f4_l3,f5_l1,f5_l2,,,
15.12.2018,2018,12,15,Wilbur,yes,metropolis,Germany,BadWurt,Freiburg,stationary,pen,354,23.95,2.82
14.12.2018,2018,12,14,Wilbur,no,smartpoint,Germany,Berlin,Berlin,household,wipes,971,57.05,7.60
22.10.2018,2018,10,22,Greg,yes,thebarn,Germany,Bremen,Bremen,stationary,water colours,142,21.01,3.75


- if .unstack() makes the data __wider__ then .stack() should make it __narrower__

In [15]:
# now we can use the newly created name....
s_w3 = df_w2.stack('METRICS')

In [16]:
s_w3.head(2)

ts1         f0_l1  f0_l2  f0_l3  f1      f2   f3          f4_l1    f4_l2    f4_l3     f5_l1       f5_l2  METRICS
15.12.2018  2018   12     15     Wilbur  yes  metropolis  Germany  BadWurt  Freiburg  stationary  pen    m1         354.00
                                                                                                         m2          23.95
dtype: float64

Learning: __Stacking of ALL features including metric reduces the data frame to a series, the lowest possible dimensionality !__ In this representation, every single data point corresponds to one value in the series. ALL structuring information is included in the multi-index

In [17]:
len(s_w3)

1500

### Enter the .query()-method to filter multi-indexed data:

In [18]:
# Step 1: .unstack() the category information again
#          This is important because the .query()-methods does NOT work on a series !
df_w4 = s_w3.unstack('f5_l1').fillna(0)
df_w4.head(3)

f5_l1                                                                                          appliances  \
ts1        f0_l1 f0_l2 f0_l3 f1   f2  f3         f4_l1   f4_l2 f4_l3   f5_l2          METRICS               
01.10.2018 2018  10    1     Beth yes metropolis Germany NRW   Cologne vaccum cleaner m1           469.00   
                                                                                      m2            97.67   
                                                                                      m3             0.26   

f5_l1                                                                                          household  \
ts1        f0_l1 f0_l2 f0_l3 f1   f2  f3         f4_l1   f4_l2 f4_l3   f5_l2          METRICS              
01.10.2018 2018  10    1     Beth yes metropolis Germany NRW   Cologne vaccum cleaner m1             0.0   
                                                                                      m2             0.0   
                                                                                      m3             0.0   

f5_l1                                                                                          stationary  
ts1        f0_l1 f0_l2 f0_l3 f1   f2  f3         f4_l1   f4_l2 f4_l3   f5_l2          METRICS              
01.10.2018 2018  10    1     Beth yes metropolis Germany NRW   Cologne vaccum cleaner m1              0.0  
                                                                                      m2              0.0  
                                                                                      m3              0.0

Learning: 
- Now only the categories are in the columns - not the metrics anymore. But all 3 metrics are still in the dataset, just with one entry per row. The METRICS information has basically been swapped from columns to rows.
- To achieve the STATE-level aggregation, we must still aggregate the data across the feature we want to see remain (aka  f4_l2 / "STATE"
    - METRICS is now in .index.names, not .column.names anymore
    
Next steps:
- filtering the dataset to limit it to m1-METRICS values
- aggregate data to one row per STATE (f4_l2)

In [19]:
# Step2: we want to limit the resulting data set just to unit-data,
#        so we only need the m1-metric from the newly created multi-indexed data frame
#        -> define the criteria in a list allows for quick code adjustments
c_metrics = ['m1']

In [20]:
# Step3: filter the df based on the defined criteria via the .query()-method
#        MIND the simple and straight-forward string format of the query argument !!
df_w5 = df_w4.query('METRICS == @c_metrics')
df_w5.head(3)

f5_l1                                                                                               appliances  \
ts1        f0_l1 f0_l2 f0_l3 f1    f2  f3         f4_l1   f4_l2  f4_l3      f5_l2          METRICS               
01.10.2018 2018  10    1     Beth  yes metropolis Germany NRW    Cologne    vaccum cleaner m1            469.0   
                             Darma yes care4you   Germany NRW    Düsseldorf mixer          m1             98.0   
                             Harry no  care4you   Germany Saxony Dresden    pen            m1              0.0   

f5_l1                                                                                               household  \
ts1        f0_l1 f0_l2 f0_l3 f1    f2  f3         f4_l1   f4_l2  f4_l3      f5_l2          METRICS              
01.10.2018 2018  10    1     Beth  yes metropolis Germany NRW    Cologne    vaccum cleaner m1             0.0   
                             Darma yes care4you   Germany NRW    Düsseldorf mixer          m1             0.0   
                             Harry no  care4you   Germany Saxony Dresden    pen            m1             0.0   

f5_l1                                                                                               stationary  
ts1        f0_l1 f0_l2 f0_l3 f1    f2  f3         f4_l1   f4_l2  f4_l3      f5_l2          METRICS              
01.10.2018 2018  10    1     Beth  yes metropolis Germany NRW    Cologne    vaccum cleaner m1              0.0  
                             Darma yes care4you   Germany NRW    Düsseldorf mixer          m1              0.0  
                             Harry no  care4you   Germany Saxony Dresden    pen            m1            183.0

So we solved the metrics-problem; remains the the aggregation to the state level -> groupby on f4_l2

In [21]:
l_row_agg = ['f4_l2']

In [22]:
# now we have everything we need for the aggregation:
df_w6 = df_w5.groupby(level = l_row_agg).sum()

In [23]:
df_w6

f5_l1,appliances,household,stationary
f4_l2,,,
BadWurt,10624.0,15941.0,16994.0
Bavaria,10421.0,11356.0,24719.0
Berlin,4970.0,2169.0,4246.0
Bremen,5745.0,4665.0,8792.0
Hamburg,963.0,4564.0,3732.0
Hessia,4528.0,3176.0,7719.0
NRW,24761.0,15041.0,26136.0
Saxony,4916.0,10055.0,17640.0



#### ....et voilà !
if we want any other filtering, it's easy to accomplish: just __adjust the filter criteria and the surprisingly straight-forward logic expressed in the .query()-method__
 - e.g. only get data for the customer care4you [f3] in Bavaria and NRW [f4_l2]

In [24]:
# Step1: define criteria for metrics
c_metrics = ['m1']
c_f3 = ['care4you']
c_f4_l2 = ['Bavaria', 'NRW']

In [25]:
# Step2: filter the df based on the defined criteria via the .query()-method
df_w7 = df_w4.query('METRICS == @c_metrics & f3 == @c_f3 & f4_l2 == @c_f4_l2')

In [26]:
# Step3: GroupBy-aggregation
df_w8 = df_w7.groupby(level = l_row_agg).sum()
df_w8

f5_l1,appliances,household,stationary
f4_l2,,,
Bavaria,3631.0,1993.0,4408.0
NRW,3810.0,2773.0,6431.0


All we have seen so far can be done easily in Excel (filtering can become a tedious point&click-orgy very quickly, though). Let us now concentrate on things that Excel cannot do.

## Where Excel fails part 1: adding calculations

- It's next to impossible to use classical functions to perform caculations on values in an Excel Pivot table. Most of the time, this is done by doing a "copy&paste_value" of the dynamic pivot table.
- This is way more easy in Pandas:
    - example use case: We want to compare the unit sales performance per category and product in Bavaria vs. NRW

In [27]:
df_w9 = s_w3.unstack('f0_l1').fillna(0)
# This is a trick to transform the series to a dataframe with a single column
# so that the .query()-method can be used.
# Incidently, there is only ONE categorical value for the feature f0_l1 in the entire dataset
# as all data is from 2018.
# It's never a bad idea to have one feature with the same value for ALL data points
# in a TLF-dataset :-)
df_w9.head(3)

f0_l1                                                                                                 2018
ts1        f0_l2 f0_l3 f1   f2  f3         f4_l1   f4_l2 f4_l3   f5_l1      f5_l2          METRICS        
01.10.2018 10    1     Beth yes metropolis Germany NRW   Cologne appliances vaccum cleaner m1       469.00
                                                                                           m2        97.67
                                                                                           m3         0.26

In [28]:
# Step1: define criteria for metrics
c_metrics = ['m1']
c_f4_l2 = ['Bavaria', 'NRW']

In [29]:
# Step2: filter the df based on the defined criteria via the .query()-method
df_w10 = df_w9.query('METRICS == @c_metrics & f4_l2 == @c_f4_l2')
df_w10.head(3)

f0_l1                                                                                                     2018
ts1        f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l2 f4_l3      f5_l1      f5_l2          METRICS       
01.10.2018 10    1     Beth   yes metropolis Germany NRW   Cologne    appliances vaccum cleaner m1       469.0
                       Darma  yes care4you   Germany NRW   Düsseldorf appliances mixer          m1        98.0
                       Wilbur no  care4you   Germany NRW   Essen      household  wipes          m1       918.0

In [30]:
# Step3: re-assign columns: 2018 [f0_l1] goes back to rows, state [f4_l2] goes to columns
s_w11 = df_w10.stack('f0_l1')
df_w12 = s_w11.unstack('f4_l2').fillna(0)
df_w12.head(3)

f4_l2                                                                                                    Bavaria  \
ts1        f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l3      f5_l1      f5_l2          METRICS f0_l1            
01.10.2018 10    1     Beth   yes metropolis Germany Cologne    appliances vaccum cleaner m1      2018       0.0   
                       Darma  yes care4you   Germany Düsseldorf appliances mixer          m1      2018       0.0   
                       Wilbur no  care4you   Germany Essen      household  wipes          m1      2018       0.0   

f4_l2                                                                                                      NRW  
ts1        f0_l2 f0_l3 f1     f2  f3         f4_l1   f4_l3      f5_l1      f5_l2          METRICS f0_l1         
01.10.2018 10    1     Beth   yes metropolis Germany Cologne    appliances vaccum cleaner m1      2018   469.0  
                       Darma  yes care4you   Germany Düsseldorf appliances mixer          m1      2018    98.0  
                       Wilbur no  care4you   Germany Essen      household  wipes          m1      2018   918.0

In [31]:
# Step4: aggregation along the desired features category & products
l_row_agg = ['f5_l1', 'f5_l2']
df_w13 = df_w12.groupby(level = l_row_agg).sum()
df_w13

f4_l2                       Bavaria      NRW
f5_l1      f5_l2                            
appliances micro wave oven   2603.0   7278.0
           mixer             4083.0   6879.0
           vaccum cleaner    3735.0  10604.0
household  gloves             939.0   5382.0
           towls             3337.0   1723.0
           waste bags        4235.0   2892.0
           wipes             2845.0   5044.0
stationary ball-pen          6502.0   3853.0
           marker            6177.0   4391.0
           pen               4359.0   5855.0
           pencil            5239.0   3808.0
           water colours     2442.0   8229.0

Now we want to compare the figures "NRW vs. Bavaria", both as absolute delta and as percentage difference:

In [32]:
df_w13 = df_w13.assign(DELTAabs = lambda x: x.NRW - x.Bavaria)
df_w13 = df_w13.assign(DELTAper = lambda x: ((x.NRW / x.Bavaria)-1)*100)

# df_w13['DELTAabs'] = df_w13['NRW']-df_w13['Bavaria']
# df_w13['DELTAper'] = ((df_w13['NRW']/df_w13['Bavaria'])-1)*100

df_w13

f4_l2                       Bavaria      NRW  DELTAabs    DELTAper
f5_l1      f5_l2                                                  
appliances micro wave oven   2603.0   7278.0    4675.0  179.600461
           mixer             4083.0   6879.0    2796.0   68.479060
           vaccum cleaner    3735.0  10604.0    6869.0  183.908969
household  gloves             939.0   5382.0    4443.0  473.162939
           towls             3337.0   1723.0   -1614.0  -48.366797
           waste bags        4235.0   2892.0   -1343.0  -31.711924
           wipes             2845.0   5044.0    2199.0   77.293497
stationary ball-pen          6502.0   3853.0   -2649.0  -40.741310
           marker            6177.0   4391.0   -1786.0  -28.913712
           pen               4359.0   5855.0    1496.0   34.319798
           pencil            5239.0   3808.0   -1431.0  -27.314373
           water colours     2442.0   8229.0    5787.0  236.977887

In [33]:
df_w13.columns.names
# df_w13.index.names

FrozenList(['f4_l2'])

In [34]:
df_w13.columns.values
#df_w13.index.values

array(['Bavaria', 'NRW', 'DELTAabs', 'DELTAper'], dtype=object)

## New METRICS can be calculated for the entire data set, e.g. revenue as METRIC m4

....you may have noticed that the metrics did contain a unit and a price info, but no revenue info, so it may be worth to add it.

In [35]:
df_w2.head(3)

,,,,,,,,,,,METRICS,m1,m2,m3
ts1,f0_l1,f0_l2,f0_l3,f1,f2,f3,f4_l1,f4_l2,f4_l3,f5_l1,f5_l2,,,
15.12.2018,2018,12,15,Wilbur,yes,metropolis,Germany,BadWurt,Freiburg,stationary,pen,354,23.95,2.82
14.12.2018,2018,12,14,Wilbur,no,smartpoint,Germany,Berlin,Berlin,household,wipes,971,57.05,7.60
22.10.2018,2018,10,22,Greg,yes,thebarn,Germany,Bremen,Bremen,stationary,water colours,142,21.01,3.75


In [36]:
# calculate revenue metric:
df_w20 = df_w2.assign(m4 = lambda x: x.m1 * x.m2)

In [37]:
df_w20.head(2)

,,,,,,,,,,,METRICS,m1,m2,m3,m4
ts1,f0_l1,f0_l2,f0_l3,f1,f2,f3,f4_l1,f4_l2,f4_l3,f5_l1,f5_l2,,,,
15.12.2018,2018,12,15,Wilbur,yes,metropolis,Germany,BadWurt,Freiburg,stationary,pen,354,23.95,2.82,8478.30
14.12.2018,2018,12,14,Wilbur,no,smartpoint,Germany,Berlin,Berlin,household,wipes,971,57.05,7.60,55395.55


__Comparing  Bavaria and NRW in terms of revenue per product category (and not as previously, units)__ is now essentially the same calculation, just on a different metric (m4 instead of m1)

In [38]:
# Step1: define the different criteria:
c_metrics = ['m4']   ###essentially the only difference to the previous code
c_f4_l2 = ['Bavaria', 'NRW']
l_row_agg = ['f5_l1', 'f5_l2'] # Product Category and Product information to aggregate the info on

In [39]:
# Step2: perform the same steps as above
s_w4 = df_w20.stack('METRICS') # move metrics to rows and reduce data set to series by doing so

In [40]:
df_w20 = s_w4.unstack('f0_l1').fillna(0) # move YEAR-info to only column; works because all values are "2018"

In [41]:
df_w21 = df_w20.query('METRICS == @c_metrics & f4_l2 == @c_f4_l2') # perform filtering on DF according to criteria

In [42]:
df_w21 = df_w21.stack('f0_l1')  # move YEAR-info back to rows (...makes df_w21 technically a Series for a moment...)
df_w21 = df_w21.unstack('f4_l2').fillna(0) # move (filtered) STATE info columns

In [43]:
df_w21 = df_w21.groupby(level = l_row_agg).sum() # perform aggregation on Product Category and Product

In [44]:
# calculate the differential
df_w21 = df_w21.assign(DELTAabs = lambda x: x.NRW - x.Bavaria)
df_w21 = df_w21.assign(DELTAper = lambda x: ((x.NRW / x.Bavaria)-1)*100)
df_w21

f4_l2                         Bavaria        NRW   DELTAabs    DELTAper
f5_l1      f5_l2                                                       
appliances micro wave oven   99429.73  293827.84  194398.11  195.513062
           mixer            163887.50  326749.03  162861.53   99.373979
           vaccum cleaner   229170.52  490948.38  261777.86  114.228418
household  gloves            78166.48  311737.08  233570.60  298.811716
           towls            221031.49   67503.50 -153527.99  -69.459781
           waste bags       232631.54  105732.58 -126898.96  -54.549336
           wipes             77680.24  296930.76  219250.52  282.247480
stationary ball-pen         370317.46  262883.77 -107433.69  -29.011241
           marker           318396.12  159287.73 -159108.39  -49.971837
           pen              287869.52  239729.53  -48139.99  -16.722851
           pencil           252545.44  186248.26  -66297.18  -26.251585
           water colours    119757.25  358144.77  238387.52  199.058946

...et voilà: the same info, but now for the revenue instead of units

# Where Excel fails Part2: producing TLF-format from csv-input

## Typical open data sources will allow multidimensional export
For example Eurostat (https://ec.europa.eu/eurostat/data/database#) from which the Table educ_uoe_perp01 was exported (number of teaching personal per country broken down per taught eduction level and sex)
### The typical export will look like this:


In [45]:
%%html
<img src="img/EuroStat_Export_Format.PNG">

##### After eliminating the first 9 rows, the import of this data via CSV will look like this:

In [46]:
df_eurostat_edu_standard = pd.read_csv('educ_uoe_perp01_export.csv',  sep=',', decimal =',')
df_eurostat_edu_standard.head(3)

,Unnamed: 0,SEX,M,M.1,M.2,M.3,M.4,F,F.1,F.2,F.3,F.4
0,ISCED11,GEO/TIME,2013,2014,2015,2016,2017,2013,2014,2015,2016,2017
1,ED0,BE,1.107,1.113,1.164,1.197,1.181,32.546,33.352,34.090,34.429,34.709
2,ED0,BG,43,44,55,47,:,18.466,18.725,18.927,18.810,:


##### ...which looks like a lot of data prep  before we can start with the actual work!!!
- e.g. the two sex categories (M/F) were interpretated as column heads and had a .#-suffix added for uniqueness
- the year-info is not correctly interpretated, because there can only be one column name

## But with very little work on the csv-format, the data can be used directly

### The modified csv-file allows Pandas, together with the right parameter in the .pd.read_csv import method, to correctly interprete the data-structure as multi-index:


In [47]:
%%html
<img src="img/EuroStat_Pandas_MultiIndex_Format.PNG">

In [48]:
###################################################################
# Probably the most important takeaway from this notebook:
# Providing the right import format from the start dramatically
# reduces data preparation time and complexity.
# Complex multidimensional data can be imported as multi-indexed
# data frame with the 'header' and 'index_col' parameter of the
# .pd.read_csv-method
# .......try to do that with Excel !!!!!!!
###################################################################

df_eurostat_edu_multiindex = pd.read_csv('educ_uoe_perp01_pandas_multi_index.csv', 
                                         sep=',', decimal =',', header=[0,1], index_col=[0,1])
                                        # header: column headers (names in green)
                                        # index_col: columns serving to index rows (names in orange)
df_eurostat_edu_multiindex.head(3)

SEX                 M                                   F                  \
TIME             2013   2014   2015   2016   2017    2013    2014    2015   
EDU_LEVEL CTRY                                                              
ED0       BE    1.107  1.113  1.164  1.197  1.181  32.546  33.352  34.090   
          BG       43     44     55     47      :  18.466  18.725  18.927   
          CZ      123    103    112    125      :  25.656  26.778  27.512   

SEX                             
TIME              2016    2017  
EDU_LEVEL CTRY                  
ED0       BE    34.429  34.709  
          BG    18.810       :  
          CZ    27.640       :

In [49]:
df_eurostat_edu_multiindex.columns.names
# df_eurostat_edu_multiindex.index.names
# df_eurostat_edu_multiindex.columns.values
# df_eurostat_edu_multiindex.index.values

FrozenList(['SEX', 'TIME'])

In [50]:
df_eurostat_edu_standard.columns.names
# df_eurostat_edu_standard.index.names
# df_eurostat_edu_standard.columns.values
# df_eurostat_edu_standard.index.values

FrozenList([None])

In [51]:
df_eurostat_edu_standard.columns

Index(['Unnamed: 0', 'SEX', 'M', 'M.1', 'M.2', 'M.3', 'M.4', 'F', 'F.1', 'F.2',
       'F.3', 'F.4'],
      dtype='object')

### Stacking and unstacking will work directly on this data set:

In [52]:
df_w30 = df_eurostat_edu_multiindex.stack('TIME')
df_w30.head(10)

SEX                       F      M
EDU_LEVEL CTRY TIME               
ED0       BE   2013  32.546  1.107
               2014  33.352  1.113
               2015  34.090  1.164
               2016  34.429  1.197
               2017  34.709  1.181
          BG   2013  18.466     43
               2014  18.725     44
               2015  18.927     55
               2016  18.810     47
               2017       :      :

# Conclusion

## Excel:
- Excel unbeatable for quick ad-hoc analysis if data is available in TLF
- Excel's problems start with
    - workload from data preparation for non-TLF-data
    - data size (especially if columns are based on functions: "vlookup()-hell")
    - the need for downstream manipulation of the pivot table results

## Python with Pandas:
Elegantly addresses Excel's shortcomings
- Performance of stack/unstack on large data sets
- Elegant import of non-TLF-data for easy data preparation / generation of TLF-format
- Easy downstream manipulation
        - data enhancements (e.g. new features based on existing features like e.g. binning)
        - calculations performed on aggregates
- Easy reversing of manipulated data to TLF
- Beyond data issue: seperation of data and logic (code) makes results easier to audit an errors easier to track



### Shortcomings of Python and Pandas / warnings to keep in mind:
- No automatic aggegation
    - but wider range of potential aggregation functions
- Multi-Index requires UNIQUE combination of features in index
    - prior group-by across all index levels solves this issue
- Some operations need sorted levels
    - see https://www.somebits.com/~nelson/pandas-multiindex-slice-demo.html for details
        - The simple solution is to sort the DataFrame using sortlevel(), which sorts lexicographically. If you don't like that sorting order, more control is available with sort_index(). 
        - code: df.sortlevel(inplace=True)

### More links:
- Easy intro and overview on Pandas
    - Link towardsdatascience:
        - https://towardsdatascience.com/be-a-more-efficient-data-scientist-today-master-pandas-with-this-guide-ea362d27386
- Working with Excel in Python
    - Link datasciencecentral:
        - https://www.datasciencecentral.com/profiles/blogs/tools-for-working-with-excel-and-python
    - ...and, of course:
        - https://openpyxl.readthedocs.io/en/stable/

# THE END